In [1]:
# ==========================================================
# UIDAI PROJECT — UNIFIED SETUP CELL (DO NOT MODIFY)
# ==========================================================

from pathlib import Path
import pandas as pd

# ---------- PATH RESOLUTION ----------
PROJECT_ROOT = Path.cwd().parent

DATA_DIR   = PROJECT_ROOT / "data"
ENROL_DIR  = DATA_DIR / "enrolment"
DEMO_DIR   = DATA_DIR / "demographic"
BIO_DIR    = DATA_DIR / "biometric"
OUTPUT_DIR = PROJECT_ROOT / "outputs"

for p in [ENROL_DIR, DEMO_DIR, BIO_DIR]:
    if not p.exists():
        raise FileNotFoundError(f"Missing data directory: {p}")

print("📁 Project root resolved:", PROJECT_ROOT)

# ---------- DATA LOADER ----------
def load_dataset(folder: Path, pattern: str):
    files = sorted(folder.glob(pattern))
    if not files:
        raise ValueError(f"No files found in {folder} matching {pattern}")
    print(f"📄 Loading {len(files)} files from {folder.name}")
    return pd.concat(
        [pd.read_csv(f, low_memory=False) for f in files],
        ignore_index=True
    )

enrolment_full   = load_dataset(ENROL_DIR, "api_data_aadhar_enrolment_*.csv")
demographic_full = load_dataset(DEMO_DIR, "api_data_aadhar_demographic_*.csv")
biometric_full   = load_dataset(BIO_DIR, "api_data_aadhar_biometric_*.csv")

# ---------- DATE NORMALISATION ----------
for name, df in {
    "enrolment": enrolment_full,
    "demographic": demographic_full,
    "biometric": biometric_full
}.items():
    
    df.columns = [c.strip().lower() for c in df.columns]
    
    df["date"] = pd.to_datetime(
        df["date"],
        dayfirst=True,    # DD-MM-YYYY
        errors="coerce"
    )
    
    print(
        f"🕒 {name} | dtype:", df["date"].dtype,
        "| null dates:", df["date"].isna().sum()
    )

# ---------- SAFETY ASSERTIONS ----------
assert enrolment_full["date"].dtype == "datetime64[ns]"
assert demographic_full["date"].dtype == "datetime64[ns]"
assert biometric_full["date"].dtype == "datetime64[ns]"

print("✅ Unified setup completed successfully")


📁 Project root resolved: d:\MSc. Data Science\SEM - 4\UDAI HACKATHON\UIDAI_Operational_Stress
📄 Loading 3 files from enrolment
📄 Loading 5 files from demographic
📄 Loading 4 files from biometric
🕒 enrolment | dtype: datetime64[ns] | null dates: 0
🕒 demographic | dtype: datetime64[ns] | null dates: 0
🕒 biometric | dtype: datetime64[ns] | null dates: 0
✅ Unified setup completed successfully


In [2]:
start = pd.to_datetime("2025-04-01")
end   = pd.to_datetime("2025-10-31")

enrol_time = enrolment_full[(enrolment_full["date"] >= start) & (enrolment_full["date"] <= end)]
demo_time  = demographic_full[(demographic_full["date"] >= start) & (demographic_full["date"] <= end)]
bio_time   = biometric_full[(biometric_full["date"] >= start) & (biometric_full["date"] <= end)]


In [3]:
common_dates = (
    set(enrol_time["date"].dropna().unique())
    & set(demo_time["date"].dropna().unique())
    & set(bio_time["date"].dropna().unique())
)

print("Common reporting days:", len(common_dates))


Common reporting days: 42


In [4]:
enrol_geo = enrol_time[enrol_time["date"].isin(common_dates)]
demo_geo  = demo_time[demo_time["date"].isin(common_dates)]
bio_geo   = bio_time[bio_time["date"].isin(common_dates)]


In [5]:
def agg_by_district(df):
    age_cols = [c for c in df.columns if "age" in c.lower()]
    return (
        df.groupby("district")[age_cols]
          .sum()
          .sum(axis=1)
          .reset_index(name="total")
    )

enrol_dist = agg_by_district(enrol_geo).rename(columns={"total": "enrolment_total"})
demo_dist  = agg_by_district(demo_geo).rename(columns={"total": "demographic_total"})
bio_dist   = agg_by_district(bio_geo).rename(columns={"total": "biometric_total"})


In [6]:
def agg_by_district(df):
    age_cols = [c for c in df.columns if "age" in c.lower()]
    return (
        df.groupby("district")[age_cols]
          .sum()
          .sum(axis=1)
          .reset_index(name="total")
    )

enrol_dist = agg_by_district(enrol_geo).rename(columns={"total": "enrolment_total"})
demo_dist  = agg_by_district(demo_geo).rename(columns={"total": "demographic_total"})
bio_dist   = agg_by_district(bio_geo).rename(columns={"total": "biometric_total"})


In [7]:
geo_filtered = (
    enrol_dist
    .merge(demo_dist, on="district")
    .merge(bio_dist, on="district")
)

# remove low-denominator noise
geo_filtered = geo_filtered[geo_filtered["enrolment_total"] >= 100]

print("Districts retained:", geo_filtered.shape[0])


Districts retained: 806


In [8]:
demo_thresh = geo_filtered["demographic_total"].median() / geo_filtered["enrolment_total"].median()
bio_thresh  = geo_filtered["biometric_total"].median() / geo_filtered["enrolment_total"].median()

geo_filtered["demo_to_enrol_ratio"] = geo_filtered["demographic_total"] / geo_filtered["enrolment_total"]
geo_filtered["bio_to_enrol_ratio"]  = geo_filtered["biometric_total"] / geo_filtered["enrolment_total"]

def classify_stress(row):
    if row["demo_to_enrol_ratio"] >= demo_thresh and row["bio_to_enrol_ratio"] >= bio_thresh:
        return "High Demographic + High Biometric"
    elif row["demo_to_enrol_ratio"] >= demo_thresh:
        return "High Demographic Stress"
    elif row["bio_to_enrol_ratio"] >= bio_thresh:
        return "High Biometric Stress"
    else:
        return "Low Stress"

geo_filtered["stress_type"] = geo_filtered.apply(classify_stress, axis=1)

geo_filtered["stress_type"].value_counts()


stress_type
Low Stress                           291
High Demographic + High Biometric    260
High Demographic Stress              149
High Biometric Stress                106
Name: count, dtype: int64

In [9]:
# Contribution by stress type
stress_contrib = (
    geo_filtered
    .groupby("stress_type")[["demographic_total", "biometric_total"]]
    .sum()
)

# Convert to percentage contribution
stress_contrib_pct = (
    stress_contrib / stress_contrib.sum()
) * 100

stress_contrib_pct


,demographic_total,biometric_total
stress_type,,
High Biometric Stress,5.931946,14.190622
High Demographic + High Biometric,33.227607,38.106933
High Demographic Stress,31.521480,16.812679
Low Stress,29.318967,30.889765


In [ ]:
from pathlib import Path

# Ensure processed data directory exists
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Alias the final risk dataframe used for escalation / predictive insight
# Update this variable if you later rename or extend the final dataframe.
final_risk_df = geo_filtered.copy()

output_path = PROCESSED_DIR / "final_risk_scores.csv"
final_risk_df.to_csv(output_path, index=False)

print(f"✅ Final risk scores exported to: {output_path}")